In [1]:
from sentence_transformers import SentenceTransformer, util
import numpy as np
def read_lines_from_files(file_paths):
    all_lines = []
    for file_path in file_paths:
        with open(file_path, 'r', encoding='utf-8') as file:
            lines = file.readlines()
            all_lines.extend([line.strip() for line in lines])
    return all_lines


train_paths = [ "dialogue_loan.txt", "dialogue_org.txt"]
test_path = ["dialogue.txt"]

# Corpus with example sentences
corpus = read_lines_from_files(train_paths)
#print(corpus)
embedder = SentenceTransformer("jhgan/ko-sroberta-multitask")

corpus_embeddings = embedder.encode(corpus, convert_to_tensor=True)

/Users/nikejune/anaconda3/envs/lv/lib/python3.8/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
2024-06-15 07:33:47.947063: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/Users/nikejune/anaconda3/envs/lv/lib/python3.8/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [3]:
# Query sentences:
queries = read_lines_from_files(test_path)[5:10]

# Find the closest 5 sentences of the corpus for each query sentence based on cosine similarity
top_k = 5
for query in queries:
 query_embedding = embedder.encode(query, convert_to_tensor=True)
 cos_scores = util.pytorch_cos_sim(query_embedding, corpus_embeddings)[0]
 cos_scores = cos_scores.cpu()

 #We use np.argpartition, to only partially sort the top_k results
 top_results = np.argpartition(-cos_scores, range(top_k))[0:top_k]

 print("\n\n======================\n\n")
 print("Query:", query)
 print("\nTop 5 most similar sentences in corpus:")

 for idx in top_results[0:top_k]:
  print(corpus[idx].strip(), "(Score: %.4f)" % (cos_scores[idx]))





Query: 알고 계신 사실에 대해서 은닉 또는 허위 진술을 해서도 안되고요.

Top 5 most similar sentences in corpus:
거짓말을 하시거나 숨기시면 안 되시고요. (Score: 0.8078)
응, 예 거짓말하시거나 숨기시면 안 됩니다. (Score: 0.7933)
따라서 본인께서 거짓말을 하시거나 은닉을 하시면 안 되시고 아시는 내용 있으면 저희 쪽으로 솔직히 말씀해 주세요. (Score: 0.7901)
본인께서는 거짓말을 하시거나 숨기시면 절대 안 되시고요. (Score: 0.7753)
이런 부분에 대해서 발설을 하시면 안 돼요. (Score: 0.7600)




Query: 네 지금 계시는 장소는 어디시죠.

Top 5 most similar sentences in corpus:
지금 위치하신 곳이 어디세요. (Score: 0.9153)
네 지금 어디시라구요. (Score: 0.8751)
지금 어디시라고요. (Score: 0.8623)
지금 네 지금 어디에 계신 거예요. (Score: 0.8355)
위치가 어디쯤 됩니까? (Score: 0.8147)




Query: 지금 카페에 있는데요.

Top 5 most similar sentences in corpus:
예 카페예요. (Score: 0.8658)
아 지금 커피숍이에요. (Score: 0.7502)
카페 통에 있는데, 카페를 통해서 했다고요. (Score: 0.7462)
지금 근처 카페에 출근하려러 나갈 거거든요. (Score: 0.6950)
네, 네 지금 주변이 좀 소음이라든가 잡음이 좀 더 심한 아 잠시만요 저는 커피숍에서 혼자 지금 과제를 하고 있었어 가지구 아 커피숍 안에 있으신 건가요? (Score: 0.6132)




Query: 본인이 이 사건에 대해 전혀 관여된 바 같고, 피해자이시라면 있는 그대로 거짓 없이 사실 그대로만 진술을 하시면 됩니다.

Top 5 most similar sentences in corpus:
네 본인께서 진술하신 내용